In [1]:
import ipywidgets as widgets

In [2]:
widgets.IntSlider()

IntSlider(value=0)

In [70]:
f = widgets.FileUpload(accept='image/*', multiple=False)
display(f)

FileUpload(value={}, accept='image/*', description='Upload')

In [88]:
w = widgets.Widget()

w.value = f.data[0]

In [95]:
widgets.jslink((f, 'data'), (i, 'value'))

Link(source=(FileUpload(value={'Screenshot_2020-03-17_17-11-49.png': {'metadata': {'name': 'Screenshot_2020-03…